# Number Detector

The goal of this little Project is to get familiar with PyTorch to create machine learning models. For this simple introduction I am going to use the following Dataset of hand painted Numbers between 0 and 9, to Train Test and Validate the MLM.

Dataset: https://github.com/SebLague/Mnist-data-numpy-format

In [5]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

netPath = "../models/numberDetection.pth"
dataPath = "../data/Mnist-HanddrawnNumbers-Dataset/mnist.npz"
data = np.load(dataPath)

trainImg = data["training_images"]
trainLbl = data["training_labels"]
testImg = data["test_images"]
testLbl = data["test_labels"]
valiImg = data["validation_images"]
valiLbl = data["validation_labels"]

t_trainImg = torch.tensor(trainImg)
t_trainLbl = torch.tensor(np.reshape(trainLbl, [50000,10]))   
t_testImg = torch.tensor(testImg)
t_testLbl = torch.tensor(np.reshape(testLbl, [10000, 10]))
t_valiImg = torch.tensor(valiImg)
t_valiLbl = torch.tensor(np.reshape(valiLbl, [10000, 10]))

trainDS = DataLoader(TensorDataset(t_trainImg, t_trainLbl))
testDS = DataLoader(TensorDataset(t_testImg, t_testLbl))
valiDS = DataLoader(TensorDataset(t_valiImg, t_valiLbl))

## Creating the Neural Network

The next step after Loading the Data is to create the Layers of the Neural Network

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatt = nn.Flatten()
        self.lrs = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )
        self.double()

    def forward(self, elem):
        elem = torch.flatten(elem)
        out = self.lrs(elem)
        return F.normalize(out, dim=0)
    
net = MyNet()



## Training

In [11]:
from tqdm import tqdm
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), 0.01, 0.5)
celoss = nn.CrossEntropyLoss()

EPOCHS = 2
for e in range(EPOCHS):
    loss = 0.0
    for i, d in enumerate(tqdm(trainDS, f"EPOCH: ({e + 1}/{EPOCHS})")):
        img, lbl = d
        optimizer.zero_grad()
        out = net.forward(img)
        loss = celoss(out, lbl[0])
        loss.backward()
        optimizer.step()

torch.save(net.state_dict(), netPath)


EPOCH: (2/2): 100%|██████████| 50000/50000 [02:11<00:00, 378.79it/s]


## Testing the Network

Hear the Network will be tested againt the Test-Set to determine its efficancy

In [35]:
net = MyNet()
net.load_state_dict(torch.load(netPath))

sum = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=float)
mask = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=float)

count = 1000

for i in range(count):
    out = net(t_valiImg[i])
    mask += t_valiLbl[i]
    out = out * t_valiLbl[i]
    sum += out

avg = sum / mask

print("Accuracy of the Digits:")
acc = 0
for i in range(10):
    a = avg[i]*100
    acc += a
    print(f"{i}: {format(a, '.2f')}%")

print(f"Total: {format(acc / 10, '.2f')}%")

Accuracy of the Digits:
0: 91.22%
1: 92.23%
2: 90.97%
3: 89.22%
4: 90.79%
5: 91.46%
6: 90.98%
7: 92.28%
8: 90.06%
9: 86.46%
Total: 90.57%


## Validating the Network

In this part a Random image of the Validation-Set will be analysed by the NN and a output is displayed to the user

In [232]:
import random

net = MyNet()
net.load_state_dict(torch.load(netPath))

def TestImage(idx):
    res = net(t_valiImg[idx])
    npres = res.detach().numpy()
    print(f"Prediction: {np.where(npres == max(npres))[0][0]} ({round(max(npres)*100, 2)}%)")
    nplbl = t_valiLbl[idx].detach().numpy()
    print(f"Value: {np.where(nplbl == 1)[0][0]}")

TestImage(random.randint(0, 10000))

Prediction: 6 (94.84%)
Value: 6
